In [1]:
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry

In [2]:
@registry.register_problem
class TRANSLATION32k(translate.TranslateProblem):

    @property
    def additional_training_datasets(self):
        """Allow subclasses to add training datasets."""
        return []

In [3]:
PROBLEM = 'translatio_n32k'
problem = problems.problem(PROBLEM)

In [4]:
import tensorflow as tf
import os

vocab_file = "t2t/data/vocab.translatio_n32k.32768.subwords"
ckpt_path = tf.train.latest_checkpoint(os.path.join('t2t/train-base'))
vocab_file, ckpt_path

('t2t/data/vocab.translatio_n32k.32768.subwords',
 't2t/train-base/model.ckpt-400000')

In [5]:
from t import text_encoder

encoder = text_encoder.SubwordTextEncoder(vocab_file)

In [6]:
class Model:
    def __init__(self, HPARAMS = "transformer_base", DATA_DIR = 't2t/data'):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        maxlen_decode = 50 + tf.reduce_max(self.X_seq_len)
        
        x = tf.expand_dims(tf.expand_dims(self.X, -1), -1)
        y = tf.expand_dims(tf.expand_dims(self.Y, -1), -1)
        
        features = {
            "inputs": x,
            "targets": y,
            "target_space_id": tf.constant(1, dtype=tf.int32),
        }
        self.features = features
        
        Modes = tf.estimator.ModeKeys
        hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_DIR, problem_name=PROBLEM)
        translate_model = registry.model('transformer')(hparams, Modes.PREDICT)
        self.translate_model = translate_model
        logits, _ = translate_model(features)
        
        with tf.variable_scope(tf.get_variable_scope(), reuse=True):
            self.fast_result = translate_model._greedy_infer(features, maxlen_decode)["outputs"]
            self.beam_result = translate_model._beam_decode_slow(
                features, maxlen_decode, beam_size=5, 
                top_beams=1, alpha=1.0)["outputs"]
        
        self.fast_result = tf.identity(self.fast_result, name = 'greedy')
        self.beam_result = tf.identity(self.beam_result, name = 'beam')
        
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, ckpt_path)

Instructions for updating:
reduction_indices is deprecated, use axis instead


Instructions for updating:
reduction_indices is deprecated, use axis instead


INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting hparams.dropout to 0.0


...

INFO:tensorflow:Building model body


INFO:tensorflow:Transforming body output with symbol_modality_26577_512.top


INFO:tensorflow:Transforming body output with symbol_modality_26577_512.top


INFO:tensorflow:Restoring parameters from t2t/train-base/model.ckpt-400000


INFO:tensorflow:Restoring parameters from t2t/train-base/model.ckpt-400000


In [7]:
string = 'KUALA LUMPUR, July 1 - Datuk Seri Anwar Ibrahim is not suitable to as the prime minister candidate as he is allegedly not "popular" among the Malays, Tun Dr Mahathir Mohamad claimed. The former prime minister reportedly said the PKR president needs someone like himself in order to acquire support from the Malays and win the election.'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0]), encoder.decode(b[0])

('KUALA LUMPUR 1 Julai - Datuk Seri Anwar Ibrahim tidak sesuai menjadi calon Perdana Menteri kerana beliau didakwa tidak "popular" dalam kalangan orang Melayu, dakwa Tun Dr. Bekas Perdana Menteri itu dilaporkan berkata presiden PKR itu memerlukan seseorang seperti dirinya demi mendapatkan sokongan daripada orang Melayu dan memenangi pilihan raya.<EOS>',
 'KUALA LUMPUR 1 Julai - Datuk Seri Anwar Ibrahim tidak sesuai menjadi calon Perdana Menteri kerana beliau didakwa tidak "popular" dalam kalangan orang Melayu, dakwa Tun Dr. Bekas Perdana Menteri itu dilaporkan berkata presiden PKR itu memerlukan seseorang seperti dirinya demi mendapatkan sokongan daripada orang Melayu dan memenangi pilihan raya.<EOS><pad><pad>')

In [8]:
string = 'Religion usually means trust in God, or a supernatural and supernatural power like God, as well as practices and institutions associated with that belief. Religion and belief are two very relevant things. But Religion has a broader meaning, which refers to a system of cohesive belief, and this belief is about the divine aspect'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0]), encoder.decode(b[0])

('Agama biasanya bermaksud kepercayaan kepada Tuhan, atau kekuatan ghaib dan ghaib seperti Tuhan, serta amalan dan institusi yang berkaitan dengan kepercayaan itu. Agama dan kepercayaan adalah dua perkara yang sangat relevan. Tetapi Agama mempunyai makna yang lebih luas, yang merujuk kepada sistem kepercayaan yang kohesif, dan kepercayaan ini adalah mengenai aspek ketuhanan<EOS>',
 'Agama biasanya bermaksud kepercayaan kepada Tuhan, atau kekuatan ghaib dan ghaib seperti Tuhan, serta amalan dan institusi yang berkaitan dengan kepercayaan itu. Agama dan kepercayaan adalah dua perkara yang sangat relevan. Tetapi Agama mempunyai makna yang lebih luas, yang merujuk kepada sistem kepercayaan yang kohesif, dan kepercayaan ini adalah mengenai aspek ketuhanan<EOS><pad><pad><pad><pad>')

In [9]:
string = 'Articulate (Well-spoken, Expressive) <> Hmmm yes Horacio Calcaterra is a sportsman that currently is playing for Sporting Cristal which is located in Torneo Descentralizado. <> He currently plays for Sporting Cristal in the Torneo Descentralizado.'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0]), encoder.decode(b[0])

('Artikulat (Baik, Ekspresif) <> Hmmm yes Horacio Calcaterra adalah olahragawan yang kini bermain untuk Sporting Cristal yang terletak di Torneo Descentralizado<> Dia kini bermain untuk Sporting Cristal dalam Torneo Descentralizado.<EOS>',
 'Artikulat (Baik, Ekspresif) <> Hmmm yes Horacio Calcaterra adalah olahragawan yang kini bermain untuk Sporting Cristal yang terletak di Torneo Descentralizado<> Dia kini bermain untuk Sporting Cristal dalam Torneo Descentralizado.<EOS><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>')

In [10]:
string = ('TANGKAK - Tan Sri Muhyiddin Yassin said he did not want to touch on '
 'political issues at the moment, instead focusing on the welfare of the '
 "people and efforts to revitalize the affected country's economy following "
 'the Covid-19 pandemic. The prime minister explained the matter when speaking '
 'at a Leadership Meeting with Gambir State Assembly (DUN) leaders at the '
 'Bukit Gambir Multipurpose Hall today.')
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0]), encoder.decode(b[0])

('TANGKAK - Tan Sri Muhyiddin Yassin berkata, beliau tidak mahu menyentuh isu politik ketika ini, sebaliknya memberi tumpuan kepada kebajikan rakyat dan usaha memulihkan ekonomi negara yang terjejas susulan pandemik Covid-19, Perdana Menteri menjelaskan perkara itu ketika berucap pada Majlis Perjumpaan Bersama Pemimpin Dewan Undangan Negeri (DUN) Gambir di Dewan Serbaguna Bukit Gambir hari ini.<EOS>',
 'TANGKAK - Tan Sri Muhyiddin Yassin berkata, beliau tidak mahu menyentuh isu politik ketika ini, sebaliknya memberi tumpuan kepada kebajikan rakyat dan usaha memulihkan ekonomi negara yang terjejas susulan pandemik Covid-19, Dr. Perdana Menteri menjelaskan perkara itu ketika berucap pada Majlis Perjumpaan Bersama Pemimpin Dewan Undangan Negeri (DUN) Gambir di Dewan Serbaguna Bukit Gambir hari ini.<EOS><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>')

In [11]:
from unidecode import unidecode

In [12]:
string = "i am in medical school."
string = unidecode(string)
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0]), encoder.decode(b[0])

('saya di sekolah perubatan.<EOS>', 'saya di sekolah perubatan.<EOS><pad>')

In [13]:
string = 'Emmerdale is the debut studio album,songs were not released in the U.S <> These songs were not released in the U.S. edition of said album and were previously unavailable on any U.S. release.'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0]), encoder.decode(b[0])

('Emmerdale adalah album studio sulung, lagu-lagu tidak dikeluarkan di A.S <> Lagu-lagu ini tidak dikeluarkan dalam album edisi AS dan sebelumnya tidak dapat digunakan pada sebarang keluaran A.S.<EOS>',
 'Emmerdale adalah album studio sulung, lagu-lagu tidak dikeluarkan di A.S <> Lagu-lagu ini tidak dikeluarkan dalam album edisi AS dan sebelumnya tidak tersedia di mana-mana keluaran A.S.<EOS><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>')

In [14]:
# https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536

string = 'Fuzzy matching at scale. From 3.7 hours to 0.2 seconds. How to perform intelligent string matching in a way that can scale to even the biggest data sets. Data in the real world is messy. Dealing with messy data sets is painful and burns through time which could be spent analysing the data itself.'
encoded = encoder.encode(string) + [1]
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: [encoded]})
encoder.decode(f[0]), encoder.decode(b[0])

('Pemadanan kabur pada skala. Dari 3.7 jam hingga 0.2 saat. Cara melakukan pemadanan rentetan pintar dengan cara yang boleh skalakan pada set data terbesar. Data dalam dunia nyata tidak kemas. Menangani set data yang tidak kemas adalah menyakitkan dan terbakar melalui masa yang mana ia boleh dibelanjakan untuk menganalisis data itu sendiri.<EOS>',
 'Pemadanan kabur pada skala. Dari 3.7 jam hingga 0.2 saat. Cara melakukan pemadanan rentetan pintar dengan cara yang boleh skalakan pada set data yang paling besar. Data dalam dunia nyata tidak kemas. Menangani set data yang tidak kemas adalah menyakitkan dan terbakar melalui masa yang boleh dibelanjakan untuk menganalisis data itu sendiri.<EOS><pad><pad><pad><pad><pad>')

In [15]:
batch_size = 256

path = 't2t/tmp/test-en'

with open(os.path.join(path, 'left.txt')) as fopen:
    left = fopen.read().split('\n')
    
with open(os.path.join(path, 'right.txt')) as fopen:
    right = fopen.read().split('\n')
    
len(left), len(right)

(66757, 66757)

In [16]:
p = sess.run(model.fast_result, feed_dict = {model.X: [encoder.encode(left[0]) + [1]]}).tolist()
results = []
for row in p:
    results.append([i for i in row if i not in [0, 1]])
results

[[78, 7003, 12422, 8151, 284, 3]]

In [17]:
from tensor2tensor.utils import bleu_hook

In [18]:
bleu_hook.compute_bleu(reference_corpus = [encoder.encode(right[0])], 
                       translation_corpus = results)

1.0

In [19]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [20]:
from tqdm import tqdm

results = []
for i in tqdm(range(0, len(left), batch_size)):
    index = min(i + batch_size, len(left))
    x = left[i: index]
    encoded = [encoder.encode(l) + [1] for l in x]
    batch_x = pad_sequences(encoded, padding='post')
    
    p = sess.run(model.fast_result, feed_dict = {model.X: batch_x}).tolist()
    result = []
    for row in p:
        result.append([i for i in row if i not in [0, 1]])
    results.extend(result)

100%|██████████| 261/261 [19:49<00:00,  4.56s/it]


In [21]:
len(results)

66757

In [22]:
rights = [encoder.encode(r) for r in right[:len(results)]]
bleu_hook.compute_bleu(reference_corpus = rights,
                       translation_corpus = results)

0.6254096

In [23]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'transformer-base/model.ckpt')

'transformer-base/model.ckpt'

In [24]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'greedy' in n.name
        or 'beam' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'modality' not in n.name
        and 'Assign' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'transformer/body/target_space_embedding/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/target_space_embedding/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_scale/Read/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_bias/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp_1',
 'transformer/body/encoder/layer_0/self_attention/multihead_attention/q/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/multihead_attention/q/Tensordot/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attenti

In [25]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [26]:
freeze_graph('transformer-base', strings)

INFO:tensorflow:Restoring parameters from transformer-base/model.ckpt


INFO:tensorflow:Restoring parameters from transformer-base/model.ckpt


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


...

INFO:tensorflow:Froze 201 variables.


INFO:tensorflow:Froze 201 variables.


INFO:tensorflow:Converted 201 variables to const ops.


INFO:tensorflow:Converted 201 variables to const ops.


11006 ops in the final graph.


In [27]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [28]:
g = load_graph('transformer-base/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
greedy = g.get_tensor_by_name('import/greedy:0')
beam = g.get_tensor_by_name('import/beam:0')
test_sess = tf.InteractiveSession(graph = g)

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [29]:
string = 'KUALA LUMPUR, July 1 - Datuk Seri Anwar Ibrahim is not suitable to as the prime minister candidate as he is allegedly not "popular" among the Malays, Tun Dr Mahathir Mohamad claimed. The former prime minister reportedly said the PKR president needs someone like himself in order to acquire support from the Malays and win the election.'
encoded = encoder.encode(string) + [1]
g, b = test_sess.run([greedy, beam], feed_dict = {x:[encoded]})

In [30]:
encoder.decode(g[0]), encoder.decode(b[0])

('KUALA LUMPUR 1 Julai - Datuk Seri Anwar Ibrahim tidak sesuai menjadi calon Perdana Menteri kerana beliau didakwa tidak "popular" dalam kalangan orang Melayu, dakwa Tun Dr. Bekas Perdana Menteri itu dilaporkan berkata presiden PKR itu memerlukan seseorang seperti dirinya demi mendapatkan sokongan daripada orang Melayu dan memenangi pilihan raya.<EOS>',
 'KUALA LUMPUR 1 Julai - Datuk Seri Anwar Ibrahim tidak sesuai menjadi calon Perdana Menteri kerana beliau didakwa tidak "popular" dalam kalangan orang Melayu, dakwa Tun Dr. Bekas Perdana Menteri itu dilaporkan berkata presiden PKR itu memerlukan seseorang seperti dirinya demi mendapatkan sokongan daripada orang Melayu dan memenangi pilihan raya.<EOS><pad><pad>')